In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("../")

import torch
from transformers import AutoTokenizer
from jiwer import wer
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger

from src.model import chained_seq2seq
from src.utils import STTDataModule

## Instantiate Model Class

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
dm = STTDataModule(device=device)
dm.setup("fit")
dm.setup("test")

train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

  0%|          | 28/10368 [02:53<17:46:26,  6.19s/it]


KeyboardInterrupt: 

In [4]:
logger = TensorBoardLogger("../model/checkpoints/tb_logs", name="chained_seq2seq", log_graph=False)
model = chained_seq2seq()

In [10]:
tokenizer_encoder = AutoTokenizer.from_pretrained("../model/opus-mt-nl-en")
model.to(device)
model.eval()

chained_seq2seq(
  (encoder): MarianMTModel(
    (model): MarianModel(
      (shared): Embedding(67028, 512, padding_idx=67027)
      (encoder): MarianEncoder(
        (embed_tokens): Embedding(67028, 512, padding_idx=67027)
        (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
        (layers): ModuleList(
          (0): MarianEncoderLayer(
            (self_attn): MarianAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation_fn): SiLUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, 

In [11]:
samp_src, samp_tgt = next(iter(test_dataloader))
src_ids = tokenizer_encoder(samp_src, return_tensors="pt", padding=True).input_ids.to(device)
tgt_ids = tokenizer_encoder(samp_tgt, return_tensors="pt",padding=True).input_ids.to(device)

In [12]:
out = model(src_ids, tgt_ids)

In [14]:
loss, logits = out[:2]
print(loss)
generated = model.generate(logits)


tensor(3.4746, device='cuda:0', grad_fn=<NllLossBackward0>)


In [17]:
wer(samp_tgt, generated)

0.44554455445544555

In [19]:
generated

['is een pizza jepk een echte pizza',
 'Demiddels waren de journalisten binnen binnen gekomen',
 'er is ook een een luchthaven en Düsseldorf',
 'Nee is mensenagenten is bijna bereik.. Ik Nee',
 'Ze was nog steeds haar h',
 'Het meisje had blond haar',
 'Ku je morgen wat te brengen? het drankborrel?',
 'Frarancoois keek de bewondering naar de documentaire',
 'Het aantal politieagenten was buiten proportie',
 'Heb jeij ooit yoga gedaan?',
 'er is ook een een luchthaven in düsseldorf',
 'Zou je dat geloven geloven? Ik',
 'is dat zelfgemaakte chocolademaak',
 'is een pizza ha her eenke echte pizza',
 'rood rood van Marccorow',
 'Blok wielentjes onder mijn bureaustoel blokkeren']

In [18]:
samp_tgt

['is een pizza hawaï een echte pizza',
 'inmiddels waren de journalisten naar binnen gekomen',
 'er is ook nog een luchthaven in düsseldorf',
 'het aantal politieagenten was buiten proportie',
 'ze zocht nog naar haar bh',
 'het meisje had blond haar',
 'kan je morgen wat eten meebrengen naar de borrel',
 'françois keek vol bewondering naar de documentaire',
 'het aantal politieagenten was buiten proportie',
 'heb jij ooit yoga gedaan',
 'er is ook nog een luchthaven in düsseldorf',
 'zou je dat wel kopen',
 'is dat zelfgemaakte chocoladesaus',
 'is een pizza hawaï een echte pizza',
 'speel rood van marco borsato',
 'de wieltjes onder mijn bureaustoel blokkeren']

In [6]:
seed_everything(42)
trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    logger=logger,
    devices=1 if torch.cuda.is_available() else None,
    default_root_dir="../model/checkpoints/"
)

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model, datamodule=dm, ckpt_path="../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints/epoch=0-step=1815.ckpt")

Restoring states from the checkpoint path at ../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints/epoch=0-step=1815.ckpt
/home/anand/Development/tinkering/stt_postprocess/venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:345: UserWarning: The dirpath has changed from '../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints' to '../model/checkpoints/tb_logs/chained_seq2seq/version_6/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | encoder   | MarianMTModel    | 79.0 M
1 | decoder   | MarianMTModel    | 79.0 M
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
156 M     Trainable params
1.0 M     Non-trainable params

Sanity Checking: 0it [00:00, ?it/s]

Training: 1815it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
